# MSA 2025 Phase 2 - Part 1

In [1]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Find all variables and understand them

In [ ]:
# Load datasets
sales = pd.read_csv('sales.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

# Display first 10 rows
print("Sales Data (first 10 rows):")
print(sales.head(10))
print("\nFeatures Data (first 10 rows):")
print(features.head(10))
print("\nStores Data (first 10 rows):")
print(stores.head(10))

# Statistical metrics
print("\nSales Data Statistics:")
print(sales.describe())
print("\nFeatures Data Statistics:")
print(features.describe())
print("\nStores Data Statistics:")
print(stores.describe())

# Visualize Weekly_Sales (example)
sns.set(style="whitegrid")
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(sales['Weekly_Sales'], bins=50, kde=True)
plt.title('Weekly Sales Distribution')
plt.subplot(1, 2, 2)
sns.boxplot(y=sales['Weekly_Sales'])
plt.title('Weekly Sales Boxplot')
plt.tight_layout()
plt.show()

# Data type conversion
sales['Date'] = pd.to_datetime(sales['Date'])
sales['IsHoliday'] = sales['IsHoliday'].astype(int)
features['Date'] = pd.to_datetime(features['Date'])
features['IsHoliday'] = features['IsHoliday'].astype(int)
for col in ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']:
    features[col] = features[col].fillna(features[col].median())
stores['Type'] = stores['Type'].map({'A': 0, 'B': 1, 'C': 2})

# Merge datasets
merged_data = sales.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
merged_data = merged_data.merge(stores, on='Store', how='left')

# Confirm data types
print("\nMerged Data Types:")
print(merged_data.dtypes)

## 2. Clean data

In [ ]:
# Load datasets
sales = pd.read_csv('sales.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

# Merge datasets
sales['Date'] = pd.to_datetime(sales['Date'])
features['Date'] = pd.to_datetime(features['Date'])
merged_data = sales.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
merged_data = merged_data.merge(stores, on='Store', how='left')

# --- Step 1: Handle missing values ---
# Reason: Missing values can degrade model performance; imputation preserves data integrity.
# MarkDown1-MarkDown5: Fill with 0 (assuming NA means no promotion)
for col in ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']:
    merged_data[col] = merged_data[col].fillna(0)
# CPI, Unemployment: Fill with median (robustness)
merged_data['CPI'] = merged_data['CPI'].fillna(merged_data['CPI'].median())
merged_data['Unemployment'] = merged_data['Unemployment'].fillna(merged_data['Unemployment'].median())
# Check missing values
print("Missing Values Statistics:")
print(merged_data.isnull().sum())

# --- Step 2: Handle outliers ---
# Reason: Outliers can distort model training; use IQR method to trim extreme Weekly_Sales.
Q1 = merged_data['Weekly_Sales'].quantile(0.25)
Q3 = merged_data['Weekly_Sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
merged_data = merged_data[(merged_data['Weekly_Sales'] >= lower_bound) & 
                          (merged_data['Weekly_Sales'] <= upper_bound)]
print(f"Data size after removing outliers: {len(merged_data)}")

# --- Step 3: Convert non-numeric columns ---
# Reason: Machine learning models require numeric input; one-hot encoding suits categorical variables.
# Type: One-hot encoding
merged_data = pd.get_dummies(merged_data, columns=['Type'], prefix='Type')
# IsHoliday: Convert to 0/1
merged_data['IsHoliday'] = merged_data['IsHoliday'].astype(int)
# Extract date features
merged_data['Year'] = merged_data['Date'].dt.year
merged_data['Month'] = merged_data['Date'].dt.month
merged_data['Week'] = merged_data['Date'].dt.isocalendar().week
merged_data = merged_data.drop('Date', axis=1)  # Remove original date column

# --- Step 4: Feature standardization ---
# Reason: Features with different scales can bias the model; standardization ensures fairness.
scaler = StandardScaler()
num_cols = ['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 
            'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size', 'Year', 'Month', 'Week']
merged_data[num_cols] = scaler.fit_transform(merged_data[num_cols])

# --- Step 5: Handle data imbalance (for IsHoliday) ---
# Reason: IsHoliday is imbalanced (fewer holiday weeks); oversampling improves model performance on holiday weeks.
X = merged_data.drop(['Weekly_Sales'], axis=1)
y = merged_data['Weekly_Sales']
# Check IsHoliday distribution
print("IsHoliday Distribution:")
print(merged_data['IsHoliday'].value_counts())
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, merged_data['IsHoliday'])
print("IsHoliday Distribution after Oversampling:")
print(pd.Series(y_resampled).value_counts())

# --- Step 6: Feature selection ---
# Reason: Irrelevant features can add noise and reduce model performance; select important features for efficiency.
selector = SelectKBest(score_func=f_regression, k=10)  # Select top 10 features
selector.fit(X, y)
selected_features = X.columns[selector.get_support()].tolist()
print("Selected Features:", selected_features)
X_selected = X[selected_features]

# --- Step 7: Save cleaned data ---
cleaned_data = pd.concat([X_selected, y], axis=1)
cleaned_data.to_csv('cleaned_data.csv', index=False)
print("Cleaned data saved as 'cleaned_data.csv'")

## 3. Visualise data

In [ ]:
# Load cleaned data
data = pd.read_csv('cleaned_data.csv')

# Set plotting style
sns.set(style="whitegrid")

# --- Visualization 1: Type (one-hot encoded as Type_A, Type_B, Type_C) ---
# Purpose: Analyze the impact of different store types on Weekly_Sales
plt.figure(figsize=(10, 6))
type_cols = ['Type_0', 'Type_1', 'Type_2']  # Type_A, Type_B, Type_C
type_sales = pd.melt(data, id_vars=['Weekly_Sales'], value_vars=type_cols, 
                     var_name='Type', value_name='IsType')
type_sales = type_sales[type_sales['IsType'] == 1]
type_sales['Type'] = type_sales['Type'].map({'Type_0': 'A', 'Type_1': 'B', 'Type_2': 'C'})
sns.boxplot(x='Type', y='Weekly_Sales', data=type_sales)
plt.title('Weekly Sales Distribution by Store Type')
plt.xlabel('Store Type')
plt.ylabel('Weekly Sales (Standardized)')
plt.show()

# --- Visualization 2: IsHoliday ---
# Purpose: Compare sales differences between holiday and non-holiday weeks
plt.figure(figsize=(8, 6))
sns.boxplot(x='IsHoliday', y='Weekly_Sales', data=data)
plt.title('Weekly Sales Distribution: Holiday vs Non-Holiday')
plt.xlabel('Is Holiday (0=Non-Holiday, 1=Holiday)')
plt.ylabel('Weekly Sales (Standardized)')
plt.show()

# --- Visualization 3: Time series aggregation (by month) ---
# Purpose: Analyze monthly sales trends
monthly_sales = data.groupby('Month')['Weekly_Sales'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(x='Month', y='Weekly_Sales', data=monthly_sales, marker='o')
plt.title('Average Weekly Sales Trend by Month')
plt.xlabel('Month')
plt.ylabel('Average Weekly Sales (Standardized)')
plt.xticks(range(1, 13))
plt.show()

# --- Visualization 4: Time series aggregation (by year) ---
# Purpose: Analyze yearly sales trends
yearly_sales = data.groupby('Year')['Weekly_Sales'].mean().reset_index()
plt.figure(figsize=(8, 6))
sns.lineplot(x='Year', y='Weekly_Sales', data=yearly_sales, marker='o')
plt.title('Average Weekly Sales Trend by Year')
plt.xlabel('Year')
plt.ylabel('Average Weekly Sales (Standardized)')
plt.xticks(yearly_sales['Year'])
plt.show()

# --- Visualization 5: Weekly and holiday interaction analysis ---
# Purpose: Explore patterns in weekly sales related to holidays
weekly_holiday_sales = data.groupby(['Week', 'IsHoliday'])['Weekly_Sales'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.lineplot(x='Week', y='Weekly_Sales', hue='IsHoliday', data=weekly_holiday_sales)
plt.title('Average Weekly Sales by Week (Holiday vs Non-Holiday)')
plt.xlabel('Week')
plt.ylabel('Average Weekly Sales (Standardized)')
plt.show()

## 4. Identify correlated variables

In [ ]:
# Load cleaned data
data = pd.read_csv('cleaned_data.csv')

# Calculate correlation matrix
correlation_matrix = data.corr(method='pearson')

# Visualize correlation matrix - heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt='.2f')
plt.title('Correlation Heatmap of Variables')
plt.show()

# Extract correlations with Weekly_Sales
weekly_sales_corr = correlation_matrix['Weekly_Sales'].sort_values(ascending=False)
print("\nCorrelations with Weekly_Sales:")
print(weekly_sales_corr)

# Identify high correlation pairs (threshold: |corr| > 0.7)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], corr_val))
print("\nHigh Correlation Pairs (|corr| > 0.7):")
for pair in high_corr_pairs:
    print(f"{pair[0]} and {pair[1]}: {pair[2]:.2f}")

## 5. Summary

For the EDA, I kicked things off by loading the sales.csv, features.csv, and stores.csv datasets, peeking at the first 10 rows to get a feel for the data. I ran stats like means and standard deviations to spot trends, then threw in histograms, boxplots, and bar charts to visualize stuff like Weekly_Sales and Type. A correlation heatmap helped me see how variables play together, especially with Weekly_Sales.

For preprocessing, I tackled missing values by filling MarkDown1-MarkDown5 with zeros (no promos, probably) and CPI and Unemployment with medians to keep things smooth. Outliers in Weekly_Sales got trimmed using IQR to avoid skewing the model. I turned Date into Year, Month, and Week, made IsHoliday a 0/1 flag, and one-hot encoded Type into Type_A, Type_B, Type_C. Numerical columns got standardized with StandardScaler for fairness, and I used SMOTE to balance the rare holiday weeks in IsHoliday. Finally, I picked the top 10 features with SelectKBest, keeping heavy hitters like Dept and Size while ditching redundant ones like Type encodings.

What stood out? Sales spike hard in November/December—think Thanksgiving and Christmas—and tank in January/February. Bigger stores (Type A) rake in more cash, while Type C lags. Holidays boost sales a bit, especially during big weeks like Black Friday. MarkDown promos don’t always pack a punch unless paired with holidays. I also noticed Type and Size were super correlated, so I dropped Type to avoid overlap. The IsHoliday imbalance was a sneaky issue, fixed with SMOTE to give holidays a fair shot. All these steps cleaned the data up nice and tight, ready for modeling!